# Phase 15: Strategy Backtests

Backtest the new modular strategies on Dec 2025 data:
- **MeanReversionVWAP**: VWAP + RSI mean reversion for XRP/USDT
- **XRPBTCPairTrading**: Cointegration-based stat arb with 10x leverage

Expected results:
- Mean reversion: +5-15% in choppy XRP $2.00-2.10 range
- Pair trading: Low-drawdown alpha, Sharpe > 1.5

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import ccxt
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Strategy imports
from strategies.mean_reversion_vwap import MeanReversionVWAP
from strategies.xrp_btc_pair_trading import XRPBTCPairTrading

plt.style.use('dark_background')
sns.set_palette('husl')

print('Imports loaded successfully')

## 1. Fetch Dec 1-7 2025 Data (1h candles)

In [ ]:
# Initialize Kraken exchange
exchange = ccxt.kraken()

def fetch_ohlcv(symbol, timeframe='1h', since=None, limit=500):
    """Fetch OHLCV data from Kraken."""
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

# Fetch Dec 1-7 2025 data
since_ts = int(datetime(2025, 12, 1).timestamp() * 1000)

print('Fetching XRP/USDT...')
xrp_df = fetch_ohlcv('XRP/USDT', '1h', since=since_ts, limit=200)

print('Fetching BTC/USDT...')
btc_df = fetch_ohlcv('BTC/USDT', '1h', since=since_ts, limit=200)

data = {
    'XRP/USDT': xrp_df,
    'BTC/USDT': btc_df
}

print(f'XRP/USDT: {len(xrp_df)} candles, {xrp_df.index[0]} to {xrp_df.index[-1]}')
print(f'BTC/USDT: {len(btc_df)} candles, {btc_df.index[0]} to {btc_df.index[-1]}')
print(f'\nXRP Price Range: ${xrp_df["close"].min():.4f} - ${xrp_df["close"].max():.4f}')
print(f'BTC Price Range: ${btc_df["close"].min():.2f} - ${btc_df["close"].max():.2f}')

## 2. Mean Reversion VWAP Backtest

In [ ]:
# Initialize strategy - PHASE 16 TUNED
mr_config = {
    'name': 'mean_reversion_vwap',
    'symbol': 'XRP/USDT',
    'vwap_window': 14,
    'rsi_window': 14,
    'dev_threshold': 0.003,  # Phase 16: 0.3% (was 0.5%)
    'rsi_oversold': 32,      # Phase 16: (was 30)
    'rsi_overbought': 68,    # Phase 16: (was 70)
    'volume_filter': True,   # Phase 16: Volume filter
    'volume_mult': 1.5,      # Phase 16: 1.5x avg volume
    'volume_window': 20,
    'max_leverage': 7,       # Phase 16: (was 5)
    'stop_loss_pct': 0.04,   # Phase 16: 4% (was 5%)
    'long_size': 0.12,
    'short_size': 0.10
}

mr_strategy = MeanReversionVWAP(mr_config)
print(f'Strategy: {mr_strategy.name} - PHASE 16 TUNED')
print(f'Config: VWAP window={mr_strategy.vwap_window}, RSI window={mr_strategy.rsi_window}')
print(f'Thresholds: RSI {mr_strategy.rsi_oversold}/{mr_strategy.rsi_overbought}, Dev {mr_strategy.dev_threshold*100:.1f}%')
print(f'Volume filter: {mr_strategy.volume_mult}x avg')
print(f'Leverage: {mr_strategy.max_leverage}x')

In [ ]:
# Run vectorized backtest
def backtest_mean_reversion(df, strategy, initial_capital=10000):
    """Backtest mean reversion strategy."""
    results = []
    capital = initial_capital
    position = 0  # 0=flat, 1=long, -1=short
    entry_price = 0
    trades = []
    
    for i in range(20, len(df)):  # Start after warmup
        # Create data slice
        data_slice = {'XRP/USDT': df.iloc[:i+1]}
        signal = strategy.generate_signals(data_slice)
        
        current_price = df['close'].iloc[i]
        timestamp = df.index[i]
        
        # Execute signals
        if signal['action'] == 'buy' and position <= 0:
            if position == -1:  # Close short
                pnl = (entry_price - current_price) / entry_price * strategy.max_leverage
                capital *= (1 + pnl * strategy.short_size)
                trades.append({'type': 'close_short', 'price': current_price, 'pnl': pnl, 'time': timestamp})
            
            # Open long
            position = 1
            entry_price = current_price
            trades.append({'type': 'long', 'price': current_price, 'time': timestamp})
            
        elif signal['action'] == 'sell' and position >= 0:
            if position == 1:  # Close long
                pnl = (current_price - entry_price) / entry_price * strategy.max_leverage
                capital *= (1 + pnl * strategy.long_size)
                trades.append({'type': 'close_long', 'price': current_price, 'pnl': pnl, 'time': timestamp})
            
            # Open short
            position = -1
            entry_price = current_price
            trades.append({'type': 'short', 'price': current_price, 'time': timestamp})
        
        results.append({
            'timestamp': timestamp,
            'price': current_price,
            'position': position,
            'capital': capital,
            'signal': signal['action'],
            'rsi': signal.get('indicators', {}).get('rsi', 50),
            'vwap': signal.get('indicators', {}).get('vwap', current_price)
        })
    
    return pd.DataFrame(results), trades

mr_results, mr_trades = backtest_mean_reversion(xrp_df, mr_strategy)
print(f'Backtest complete: {len(mr_results)} periods, {len(mr_trades)} trades')

In [ ]:
# Calculate metrics
def calculate_metrics(results_df, trades, initial_capital=10000):
    """Calculate backtest metrics."""
    final_capital = results_df['capital'].iloc[-1]
    total_return = (final_capital - initial_capital) / initial_capital * 100
    
    # Calculate daily returns for Sharpe
    results_df['returns'] = results_df['capital'].pct_change()
    sharpe = results_df['returns'].mean() / results_df['returns'].std() * np.sqrt(24 * 365) if results_df['returns'].std() > 0 else 0
    
    # Max drawdown
    results_df['peak'] = results_df['capital'].cummax()
    results_df['drawdown'] = (results_df['capital'] - results_df['peak']) / results_df['peak']
    max_dd = results_df['drawdown'].min() * 100
    
    # Win rate
    profitable_trades = [t for t in trades if t.get('pnl', 0) > 0]
    win_rate = len(profitable_trades) / max(len([t for t in trades if 'pnl' in t]), 1) * 100
    
    return {
        'Total Return': f'{total_return:.2f}%',
        'Sharpe Ratio': f'{sharpe:.2f}',
        'Max Drawdown': f'{max_dd:.2f}%',
        'Win Rate': f'{win_rate:.1f}%',
        'Trades': len([t for t in trades if 'pnl' in t]),
        'Final Capital': f'${final_capital:,.2f}'
    }

mr_metrics = calculate_metrics(mr_results, mr_trades)
print('\n=== Mean Reversion VWAP Results ===')
for k, v in mr_metrics.items():
    print(f'{k}: {v}')

In [ ]:
# Plot results
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Price with VWAP
ax1 = axes[0]
ax1.plot(mr_results['timestamp'], mr_results['price'], label='XRP Price', alpha=0.8)
ax1.plot(mr_results['timestamp'], mr_results['vwap'], label='VWAP', linestyle='--', alpha=0.7)
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.set_title('Mean Reversion VWAP Backtest')

# RSI
ax2 = axes[1]
ax2.plot(mr_results['timestamp'], mr_results['rsi'], label='RSI', color='purple')
ax2.axhline(y=70, color='red', linestyle='--', alpha=0.5, label='Overbought')
ax2.axhline(y=30, color='green', linestyle='--', alpha=0.5, label='Oversold')
ax2.set_ylabel('RSI')
ax2.legend()

# Capital
ax3 = axes[2]
ax3.plot(mr_results['timestamp'], mr_results['capital'], label='Capital', color='green')
ax3.axhline(y=10000, color='white', linestyle='--', alpha=0.3, label='Initial')
ax3.set_ylabel('Capital ($)')
ax3.set_xlabel('Date')
ax3.legend()

plt.tight_layout()
plt.show()

## 3. XRP/BTC Pair Trading Backtest

In [ ]:
# Initialize pair trading strategy - PHASE 16 TUNED
pt_config = {
    'name': 'xrp_btc_pair_trading',
    'xrp_symbol': 'XRP/USDT',
    'btc_symbol': 'BTC/USDT',
    'lookback': 168,         # Use 168 for 200-candle dataset (336 requires more data)
    'entry_z': 1.8,          # Phase 16: (was 2.0)
    'exit_z': 0.5,
    'stop_z': 3.0,
    'use_btc_filter': True,  # Phase 16: BTC momentum filter
    'btc_rsi_window': 14,
    'coint_pvalue_threshold': 0.10,  # Phase 16: Relaxed (was 0.05)
    'hedge_recalc_period': 24,
    'max_leverage': 10,
    'position_size_pct': 0.10
}

pt_strategy = XRPBTCPairTrading(pt_config)
print(f'Strategy: {pt_strategy.name} - PHASE 16 TUNED')
print(f'Lookback: {pt_strategy.lookback} periods')
print(f'Entry Z: {pt_strategy.entry_z}, Exit Z: {pt_strategy.exit_z}')
print(f'BTC Filter: {pt_strategy.use_btc_filter}, Coint threshold: {pt_strategy.coint_pvalue_threshold}')
print(f'Leverage: {pt_strategy.max_leverage}x')

In [ ]:
# Run pair trading backtest
def backtest_pair_trading(xrp_df, btc_df, strategy, initial_capital=10000):
    """Backtest pair trading strategy."""
    results = []
    capital = initial_capital
    trades = []
    
    min_len = min(len(xrp_df), len(btc_df))
    
    for i in range(strategy.lookback, min_len):
        # Create data slice
        data_slice = {
            'XRP/USDT': xrp_df.iloc[:i+1],
            'BTC/USDT': btc_df.iloc[:i+1]
        }
        
        signal = strategy.generate_signals(data_slice)
        
        xrp_price = xrp_df['close'].iloc[i]
        btc_price = btc_df['close'].iloc[i]
        timestamp = xrp_df.index[i]
        
        indicators = signal.get('indicators', {})
        zscore = indicators.get('zscore', 0)
        hedge_ratio = indicators.get('hedge_ratio', 0)
        
        # Simple PnL tracking for pair trades
        if 'close' in signal['action'] and len(trades) > 0:
            last_trade = trades[-1]
            if last_trade['action'] == 'short_xrp_long_btc':
                # Profit if XRP fell relative to BTC
                xrp_return = (last_trade['xrp_price'] - xrp_price) / last_trade['xrp_price']
                btc_return = (btc_price - last_trade['btc_price']) / last_trade['btc_price']
            else:
                # Profit if XRP rose relative to BTC
                xrp_return = (xrp_price - last_trade['xrp_price']) / last_trade['xrp_price']
                btc_return = (last_trade['btc_price'] - btc_price) / last_trade['btc_price']
            
            pnl = (xrp_return + btc_return) * strategy.max_leverage * strategy.position_size_pct
            capital *= (1 + pnl)
            trades.append({'action': 'close', 'pnl': pnl, 'time': timestamp})
        
        elif signal['action'] in ['short_xrp_long_btc', 'long_xrp_short_btc']:
            trades.append({
                'action': signal['action'],
                'xrp_price': xrp_price,
                'btc_price': btc_price,
                'zscore': zscore,
                'time': timestamp
            })
        
        results.append({
            'timestamp': timestamp,
            'xrp_price': xrp_price,
            'btc_price': btc_price,
            'zscore': zscore,
            'hedge_ratio': hedge_ratio,
            'capital': capital,
            'position': strategy.current_position,
            'signal': signal['action']
        })
    
    return pd.DataFrame(results), trades

pt_results, pt_trades = backtest_pair_trading(xrp_df, btc_df, pt_strategy)
print(f'Backtest complete: {len(pt_results)} periods, {len(pt_trades)} trades')

In [ ]:
# Calculate pair trading metrics
pt_metrics = calculate_metrics(pt_results, pt_trades)
print('\n=== XRP/BTC Pair Trading Results - PHASE 16 TUNED ===')
for k, v in pt_metrics.items():
    print(f'{k}: {v}')

# Additional pair-specific metrics (handle None values)
hedge_ratio = pt_results["hedge_ratio"].iloc[-1] if len(pt_results) > 0 else 0
zscore_min = pt_results["zscore"].min() if len(pt_results) > 0 else 0
zscore_max = pt_results["zscore"].max() if len(pt_results) > 0 else 0
coint_pval = pt_strategy.cointegration_pvalue

print(f'\nHedge Ratio (final): {hedge_ratio:.6f}')
print(f'Z-score Range: [{zscore_min:.2f}, {zscore_max:.2f}]')
print(f'Cointegration p-value: {coint_pval:.4f}' if coint_pval is not None else 'Cointegration p-value: N/A')
print(f'BTC RSI (final): {pt_strategy.btc_rsi:.1f}')

In [ ]:
# Plot pair trading results
fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

# Normalized prices
ax1 = axes[0]
xrp_norm = pt_results['xrp_price'] / pt_results['xrp_price'].iloc[0] * 100
btc_norm = pt_results['btc_price'] / pt_results['btc_price'].iloc[0] * 100
ax1.plot(pt_results['timestamp'], xrp_norm, label='XRP (normalized)', alpha=0.8)
ax1.plot(pt_results['timestamp'], btc_norm, label='BTC (normalized)', alpha=0.8)
ax1.set_ylabel('Normalized Price')
ax1.legend()
ax1.set_title('XRP/BTC Pair Trading Backtest')

# Z-score
ax2 = axes[1]
ax2.plot(pt_results['timestamp'], pt_results['zscore'], label='Z-score', color='orange')
ax2.axhline(y=2, color='red', linestyle='--', alpha=0.5, label='Entry (short XRP)')
ax2.axhline(y=-2, color='green', linestyle='--', alpha=0.5, label='Entry (long XRP)')
ax2.axhline(y=0, color='white', linestyle='-', alpha=0.3)
ax2.fill_between(pt_results['timestamp'], -0.5, 0.5, alpha=0.2, color='gray', label='Exit zone')
ax2.set_ylabel('Z-score')
ax2.legend()

# Hedge ratio
ax3 = axes[2]
ax3.plot(pt_results['timestamp'], pt_results['hedge_ratio'], label='Hedge Ratio', color='cyan')
ax3.set_ylabel('Hedge Ratio')
ax3.legend()

# Capital
ax4 = axes[3]
ax4.plot(pt_results['timestamp'], pt_results['capital'], label='Capital', color='green')
ax4.axhline(y=10000, color='white', linestyle='--', alpha=0.3, label='Initial')
ax4.set_ylabel('Capital ($)')
ax4.set_xlabel('Date')
ax4.legend()

plt.tight_layout()
plt.show()

## 4. Strategy Comparison vs Buy-Hold

In [ ]:
# Calculate buy-hold returns
xrp_buy_hold = (xrp_df['close'].iloc[-1] / xrp_df['close'].iloc[0] - 1) * 100
btc_buy_hold = (btc_df['close'].iloc[-1] / btc_df['close'].iloc[0] - 1) * 100

print('=== Strategy Comparison ===')
print(f'\nBuy-Hold XRP: {xrp_buy_hold:.2f}%')
print(f'Buy-Hold BTC: {btc_buy_hold:.2f}%')
print(f'\nMean Reversion VWAP: {mr_metrics["Total Return"]}')
print(f'XRP/BTC Pair Trading: {pt_metrics["Total Return"]}')

print('\n=== Risk-Adjusted Returns ===')
print(f'Mean Reversion Sharpe: {mr_metrics["Sharpe Ratio"]}')
print(f'Pair Trading Sharpe: {pt_metrics["Sharpe Ratio"]}')

print('\n=== Drawdowns ===')
print(f'Mean Reversion Max DD: {mr_metrics["Max Drawdown"]}')
print(f'Pair Trading Max DD: {pt_metrics["Max Drawdown"]}')

In [ ]:
# Final comparison chart
fig, ax = plt.subplots(figsize=(12, 6))

# Normalize all to 100
mr_equity = mr_results['capital'] / mr_results['capital'].iloc[0] * 100
pt_equity = pt_results['capital'] / pt_results['capital'].iloc[0] * 100
xrp_equity = xrp_df['close'] / xrp_df['close'].iloc[0] * 100

ax.plot(mr_results['timestamp'], mr_equity, label='Mean Reversion VWAP', linewidth=2)
ax.plot(pt_results['timestamp'], pt_equity, label='XRP/BTC Pair Trading', linewidth=2)
ax.plot(xrp_df.index[-len(mr_results):], xrp_equity.iloc[-len(mr_results):], 
        label='XRP Buy-Hold', linestyle='--', alpha=0.7)

ax.axhline(y=100, color='white', linestyle='--', alpha=0.3)
ax.set_xlabel('Date')
ax.set_ylabel('Equity (normalized to 100)')
ax.set_title('Strategy Comparison: Dec 2025')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('\n✓ Backtest complete! Review results above.')